In [12]:
import torch
import torchvision
from torchvision import models

In [13]:
ViT = models.vit_b_16()
ViT = models.vision_transformer._vision_transformer(
        patch_size=16,
        num_layers=12,
        num_heads=12 // 3,
        hidden_dim=768 // 3,
        mlp_dim=3072 // 3,
        weights=models.vision_transformer.ViT_B_16_Weights.verify(None),
        progress=True,
        num_classes=100
    )
ResNet = models.resnet18(pretrained=False, num_classes=100)

/root/miniconda3/envs/cc_pyg_1/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/cc_pyg_1/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [14]:
for i in range(2):
    encoderblock = list(ViT.encoder.layers)
    encoderblock.append(models.vision_transformer.EncoderBlock(
        num_heads=12 // 3,
        hidden_dim=768 // 3,
        mlp_dim=3072 // 3,
        dropout=0,
        attention_dropout=0,
        norm_layer=models.vision_transformer.partial(torch.nn.LayerNorm, eps=1e-6)
    ))
    ViT.encoder.layers = torch.nn.Sequential(*encoderblock)

In [15]:
print(ViT)

VisionTransformer(
  (conv_proj): Conv2d(3, 256, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (0): EncoderBlock(
        (ln_1): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=256, out_features=1024, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=1024, out_features=256, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (1): EncoderBlock(
        (ln_1): LayerNorm((256,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttentio

In [16]:
sum_vit = 0
params = list(ViT.parameters())
for param in params:
    p = 1
    for i in param.size():
        p *= i
    sum_vit += p
print(sum_vit)

11330404


In [17]:
sum_resnet = 0
params = list(ResNet.parameters())
for param in params:
    p = 1
    for i in param.size():
        p *= i
    sum_resnet += p
print(sum_resnet)

11227812
